First we build a group of similar business logic

Notice how the `save` and `items` methods are only
stubs. These serve as default behaviors to prevent
errors, but they aren't required

Notice how this is constructed with object syntax
rather than class syntax. This is because it's really
just a collection of logic. There's no state here really
(although there could be). The object syntax makes
extension a bit easier.

When collecting logic like this, it's better to use an
object than just a bunch of unbound functions in a module.
Objects can participate in the prototype chain, be rewritten,
or extended. Modules look like objects, but they aren't

In [2]:
const Insights = {
  async recordPageview(pageview) {
    await this?.save(pageview)
  },

  async botCount() {},

  async site() {},

  async pages() {},

  async page(path) {},

  pageviews() {
    return this.items()
  },

  async save(value) {
    console.log('Saving')
  },

  items() {
    return async function*() {}
  }
}

Here we extend the object at runtime with some effect-ful
logic. We aren't creating any new methods on the object,
just overriding existing ones.

In [3]:
Insights.store = await Deno.openKv('./insights.db')

Insights.save = async function(value) {
  await this.store.set(['insights', 'pageview', Date.now()], value)
}

Insights.items = function() {
  return this.store.list({ prefix: ['insights', 'pageview']})
}

await Insights.recordPageview({ hello: 'world' })

const pageviews = Insights.pageviews()
for await (const pageview of pageviews) {
  console.log(pageview)
}

TypeError: Deno.openKv is not a function

Alternatively, we could create a new object and extend
it with some new fields.

This prevents the Insights object from being changed
everywhere, which may not always be desirable.

Now only `KvInsights` will do effect-ful logic. But it's
still sharing all the core Insights logic

Note the `create` + `assign` setup. This is saying:
1. Create a new object based off `Insights`
2. Assign these extra properties to this new object

Note, we can't replicate this with class `extends` syntax
Insights, isn't a constructor function that makes an
object, it IS an object

In [4]:
const KvInsights = Object.assign(Object.create(Insights), {
  store: await Deno.openKv('./insights2.db'),

  async save(value) {
    await this.store.set(['insights', 'pageview', Date.now()], value)
  },

  items() {
    return this.store.list({ prefix: ['insights', 'pageview']})
  },
})

await KvInsights.recordPageview({ hello: 'world' })

const kvPageviews = KvInsights.pageviews()
for await (const pageview of kvPageviews) {
  console.log(pageview)
}

TypeError: Deno.openKv is not a function

So here's an interesting question

Do we:
1. Make `insights` a pure bag of functions, for creating
   and manipulating `Pageview` objects. And then use a
   common `store` bag of functions for `save()`-ing the
   visits
   - This how elixir does it
2. Make `Insights` "appear" to do its own persistence,
   internally but actually inject how that's done via
   the prototype chain?
   - This is how JavaScript APIs more naturally look

If I do (1), and then decide I wanna put my saving
"workflow" into a larger operation, I sorta end up
with (2) anyway; from outside observers it just looks
like a black box effectful operation. If I wanna avoid,
this, I need to make sure my `save()` call is always
at the "topmost" workflow, and that would require
refactoring my original workflow to have a 1-off
version for my original workflow, and a composable
version for when I want to put that workflow within
others. This is effectively what I did with Elixir
when working on Checkmate. Tbf it did work but I was
never fully satisfied with it, it was complicated and
I wanted to get to the level of "higher-order"
composition, rather than only composing `Multi` objects

Would the prototyping approach fare any different?

If I had an Insights + Login object, I could write say
a Server object with a workflow involving both. The
Insights logic would trigger some storage requests, as
would the Login logic. This network logic could be easily
mocked via prototype overriding, but the overall logic
wouldn't inherently be any more optimal. It'd basically
be the same in fact. Except with prototypes and method
calls I think I don't need to make a `Multi` object, which
makes for somewhat cleaner but also more implicit logic

I think prototype style may be more the way to go. I can
use it to mimic a Phoenix-like style at minimum, but in a
way that mimics standard JavaScript APIs more closely

----

Ok but maybe I should separate Visit from TimeStore anyway.
The web APIs that deal with storage are never specific to
any kind of object. There is no `PersistedDate` object,
just Date and LocalStorage. Same with network logic. Python
is this way too; it offers APIs for saving data, and
specific kinds of data, but never both at the same time

Ok this isn't necessarily true. The Navigator API is a
persisted store for all sorts of data. It is concrete, it
manages specifically user-related information, like
location or credentials. It is an opinionated, non-generic,
effect-ful store

By being more opinionated, it's less generic in the kinds
of data it can store, but it gives the data it does store
a simpler, more bespoke api by comparison

By this logic, an effectful `Insights` api is probably fine
then. It's opinionated about what it stores, but in return
there's a more specific api tailored for working with it.
If the effectful nature of it becomes a problem, that's
what I can leverage prototypes for

But well, is a more specific api better, really? I wanna
say in the case of `Navigator` yes. You could not expose
specialized apis for getting the users location, and just
expose, say, a more generic kv api, and then just say this
kv database has a "location" key.
Is that better? It's fewer new things, you just gotta
learn kv's semantics, but it's also more low-level. It
conflates location data logic with kv operations. In
theory ya I could express location logic _in terms of_ kv
operations, but that makes the higher-order location
rules more implicit, and not the main focus of the code
as it probably should be. This is the advantage of custom
semantics, it creates abstraction and lets you read the
code at a higher level.
The downside is that there's a _lot_ more to learn. When
every problem domain has it's own semantics, it means
every problem domain becomes unique, rather than sharing
common semantics where it makes sense. Moreover, you need
to learn the higher-level semantics _and_ the lower-level
ones to know if you should extend an existing high-level
api, or drop down a level and create something different
entirely. Although, I guess you'd only drop down a level
when you have to for performance reasons. Moreover, you
still need to learn the rules of data when it's written
at a lower level. Learning the rules of business logic is
easier to grasp when written in terms of high-level
operations, rather than low-level patterns

Ok so yes, higher-level operations are more specific but
easier to grasp. Lower-level operations are more generic,
but may understanding any specific operation a bit harder
to grasp. You're trading a more general solution for more
specialized understandability.

So prefer specialized semantics. Build more generalized
solutions to underpin their implementations as you develop
an understanding for what logic is common vs unique.
Specialized semantics described _in terms of_ slightly
less specialized semantics, down the chain of abstraction

If possible, you can do this in javascript via the
prototype chain, although that's just an implementation of
this idea I'm having

One issue I'm thinking about, as I've experienced working
on Discord; if you don't _know_ a system with higher-level
semantics exist, you default to using the lower-level ones
since they're more general to any kind of problem. Also
the lower you go, the fewer APIs you need to know in general
anyway, since they underpin so many things.
For example, if I didn't know the Insights API exists, I
may make, say, a `PostInsights` to give me stats on when
my posts are viewed, and for how long. It's _mostly_
similar to `Insights`, but different enough to where the
api looks a bit different under the hood. Now 2 competing
solutions exists. And even if I later discovered Insights
is a thing, I now gotta do a lot of work to merge in
my specialized version with the more general one.
How do you combat this?

Hmm, well I _guess_ in this case `PostInsights` really
is just a specialized version of `Insights`. I could change
the Post version to follow Insights in the prototype chain,
which would make the `PostInsights` implementation _much_
shorter, and probably better as well as the more general
Insights is probably better tested and well-known

Hmm, now that I think about this, this is really how it
ends up at Discord. I make a specialized thing, I realize
there's a more general pattern for what I'm doing, and then
I have to migrate to that general thing. But I could just
lump the general thing under my more specialized thing.
And if I learn my specialized thing isn't all that
different to the general thing anyway, I can just use their
more well-known semantics. But if my specialized thing
uses the more general thing in a lot of bespoke and
consistent ways, then I ya the prototype chain is the way

Damn, I really think I figured this out.

So I should just make an Insights API. Make the semantics
as specific as you need.
- If you're worried about effects being mixed with domain
  logic for practical reasons, just dependency inject fake
  versions when testing via the prototype chain.
- If you're worried about effects being mixed with domain
  logic for _conceptual_ reasons, inject abstraction
  layers in between. It's fine for Insights methods to
  call save logic internally. Each method is self-contained
  kv rules. Insights is defined _in terms of_ kv rules.
  This ensures things that wanna compose Insights logic
  with other things, like a router handler, can do so through
  higher-order Insights methods, rather than having to
  repeat raw kv operations that muddy the higher-level
  insights rule it's trying to express
- If you find you need more general semantics for your
  insights api, just make a lower-level insights api.

Contrary to what I previously believed, the answer is not
to find the most general solution to a problem. That will
also be the most low-level solution. It's ok to have a
specific solution to a specific problem. You can refactor
out and reference the more general solutions later, while
keeping your existing specific solutions as-is in the
codebase and domain specific for the localized problem at
hand

Yes a lower-level solution will involve less business
rules _as written_, but the higher-level business rules
will then just be repeated chunks of code anyway. Those
chunks are easier to read and identify when they have a
name

----

Can this really be true? Is it really that simple?

I feel like my body doesn't wanna accept this, but it
really is true I think

Also, prototypes enabled meta-programming. Or, well, `this`
enables meta-programming, because it's a contextual
keyword/object hybrid.

Imagine first an `Insights` object as is that saves to
localStorage.

Now imagine a `Login` object that also saves to localStorage

I'd like to write a workflow that combines operations from
both objects. I'd also like to optimize the writes/reads
I do to be as few as possible. For example, saving all
data between both objects in a single `set` call

With prototyping, I can have `Insights` and `Login`
delegate their internal save logic. I can then replace
that save dependency on localStorage with an object that
builds up a collection of operations (operations are now
data rather than commands). I can then create an automated
approach for "flushing" the data at some point

This is considered meta programming because my `Insights`
and `Login` logic are "programming" the localStorage object,
whatever it is, to do certain operations.

HOLY SHIT higher-level logic is "programming" lower-level
logic

----

Note, finding the right level of abstraction is really
hard. When in doubt, be concrete (aka specific). You can
always refine and generalize later.

Code should be meant to be changed, the rest of the system
should be resilient and adaptive to internal chaos and
change

I've written somewhere about how, in my experience through
Discord, I've realized code is only ever written once, and
then either deleted or kept as-is. And because of this, I
should aim for the best, most general solution to a problem
the first time around, so in case it's in the minority of
code that lives past a week, it'll stay relatively
understandable as further logic is piled on top of it.

And also so that it can be more easily re-used, rather than
reimplemented slightly differently in many different
places. If a good implementation already exists, you can
just re-use it, rather than spend the time and tech debt
re-creating and later maintaining it.

But tbf, knowing when to create vs re-use is always a hard
question, whether or not you know a general pattern exists
or not.

In actuality, I think LoC will only ever go down when you
a software engineer take a conscious effort to reduce it.
New requirements will always push LoC up. Good architecture
determines by how much at a time, but it's not something
you can stop altogether. _Only_ conscious refactoring can
reduce LoC count and complexity

Complexity is not something you can avoid. It's something
that will build up over time, that you must make a
conscious effort to reduce

You can't know the layers semantics of a system out the
gate; it's something you have to discover as it emerges
from your growing codebase

Code quality is a human problem, not a software one